In [10]:
from keras.applications import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import os
import numpy as np
from keras.preprocessing import image
from keras.utils import image_utils
import cv2
import dlib
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
# import Model
from keras.models import Model

global basedir, image_paths, target_size
basedir = os.path.dirname(os.path.abspath(''))
dataset_dir = os.path.join(basedir,'Datasets')
images_dir = os.path.join(dataset_dir,'celeba')
test_images_dir = os.path.join(dataset_dir,'celeba_test')
predictor_dir = os.path.join(basedir,'shape_predictor_68_face_landmarks.dat')
labels_filename = 'labels.csv'

base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input,      
outputs=base_model.get_layer('flatten').output)
model.summary()

553467096/553467096 [==============================] - 15s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                           

In [15]:
def get_features():
    img_path = os.path.join(images_dir,'img','1.jpg')
    x = image_utils.img_to_array(
                image_utils.load_img(img_path,
                                target_size=(224,224),
                                interpolation='bicubic'))
    print(x.shape)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    flatten = model.predict(x)
    return list(flatten[0])

features, labels = [], []

# Loop into the directory of images and extract features and labels
features.append(get_features())
features = np.array(features)
labels.append("#some label")
print(features.shape)

(224, 224, 3)
1/1 [==============================] - 0s 255ms/step
(1, 25088)


In [9]:
img_path = os.path.join(images_dir,'img','1.jpg')
x = image_utils.img_to_array(
            image_utils.load_img(img_path,
                            target_size=(224,224),
                            interpolation='bicubic'))
print(x.shape)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Extract features
features = model.predict(x)
print(features.shape)

(224, 224, 3)
1/1 [==============================] - 2s 2s/step
(1, 7, 7, 512)


In [8]:
features = features.reshape(1,)

features = features.flatten()
print(features.shape)

(15360,)


In [ ]:
print("Getting train images")
labels_df = pd.read_csv(os.path.join(images_dir, labels_filename), sep='\t')
image_paths = [os.path.join(images_dir, "img", img_name) for img_name in labels_df['img_name'].values]
all_features = np.array([])
all_labels = np.array([])

gender_labels = labels_df['gender'].values
if os.path.isdir(images_dir):
    
    for i, img_path in enumerate(image_paths):
        # load image
        x = image_utils.img_to_array(
            image_utils.load_img(img_path,
                            target_size=None,
                            interpolation='bicubic'))

        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Extract features
        features = model.predict(x)
        if features is not None:
            all_features.append(features)
            all_labels.append(gender_labels[i])